## Define Source and Target Location in S3

In [1]:
#Import Sagemaker to get excution role to read from S3
#Import Boto3 for S3 IO
#Import Pandas for Data analysis using DataFrame
from sagemaker import get_execution_role
import boto3
import pandas as pd


#Grant role to access s3
role = get_execution_role()

In [2]:
#Define Source files
logs_bucket='factorylogs'

err_codes_data_key = 'ErrorCodes.csv'
err_codes_data_location = 's3://{}/{}'.format(logs_bucket, err_codes_data_key)

event_data_key = 'OEE_Event.csv'
event_data_location = 's3://{}/{}'.format(logs_bucket, event_data_key)

In [3]:
#Define output file
output_bucket='neptunerawdata'
output_key='Raw/occurs_when.csv'
output_location = 's3://{}/{}'.format(output_bucket, output_key)

## Create Dataframe for data calculation 

In [4]:
#Create ErrorCode Dataframe 
df_error = pd.read_csv(err_codes_data_location)
df_error['Creation Time'] = pd.to_datetime(df_error['Creation Time'])
df_error.head()

,Problem,Machine,ErrorCode,Creation Time
0,bb13157f-a009-4ca2-8044-1ee80d1e4481,SMT1 Line3,30579,2022-06-09 10:44:14
1,63d20538-38e4-4189-ab5b-62328dd30a26,SMT1 Line3,30366,2022-06-09 14:25:35
2,d9f7ea74-b4a2-4bd2-86a2-1a89479200fd,SMT1 Line3,30520,2022-06-09 10:04:28
3,09857036-a781-41bb-95e0-fdba97dc3580,SMT1 Line3,30353,2022-06-09 11:57:22
4,d9f7ea74-b4a2-4bd2-86a2-1a89479257fd,SMT1 Line3,30450,2022-06-09 08:05:08


In [5]:
#Create Event Dataframe 
df_event = pd.read_csv(event_data_location)
df_event['Creation Time'] = pd.to_datetime(df_event['Creation Time'])
display(df_event)

,Event,Module,ModulePart,RootCause,RottCauseID,Creation Time
0,unplanneddown,电控系统,A3 电源板,接触不良,rc1,2022-06-09 12:40:34
1,unplanneddown,电控系统,CSB,磨损,rc2,2022-06-09 08:14:40
2,unplanneddown,贴片头,DP,松动,rc3,2022-06-09 11:19:04
3,unplanneddown,贴片头,DP传感器,变形,rc4,2022-06-09 13:07:40
4,unplanneddown,贴片头,DP过滤片,腐蚀,rc5,2022-06-09 09:06:12
...,...,...,...,...,...,...
126,unplanneddown,WPC,轴控卡,接触不良,rc127,2022-06-09 14:09:23
127,unplanneddown,PC控制系统,主板,磨损,rc128,2022-06-09 09:16:17
128,unplanneddown,设备外设,主供气管,松动,rc129,2022-06-09 10:55:20
129,unplanneddown,PC控制系统,主机电源,变形,rc130,2022-06-09 12:13:41


In [6]:
df_error['Creation Time'].describe(datetime_is_numeric=True)

count                              850
mean     2022-06-09 11:24:57.317646848
min                2022-06-09 08:00:09
25%      2022-06-09 09:43:29.250000128
50%         2022-06-09 11:18:45.500000
75%                2022-06-09 13:07:14
max                2022-06-09 14:59:21
Name: Creation Time, dtype: object

## Define data transformation logic

In [7]:
#Define dataframe_strip funtion to get ErrorCode from given time
import datetime
from datetime import timedelta

def dataframe_strip(dataframe: pd.DataFrame, timestamp: datetime,duration: timedelta):
    endtime = timestamp+duration
    outputdataframe = dataframe[(dataframe['Creation Time']>timestamp)&(dataframe['Creation Time']<endtime)]
    return outputdataframe

In [8]:
#Create ErrorCode and Event Mappings

def mapping(df_error:pd.DataFrame, df_event:pd.DataFrame):
    
    df_occurs_when = pd.DataFrame(columns = ['~id','~from', '~to','~label' ,'in30:Integer','in20:Integer','in10:Integer'])
    id = 1
    for timestamp, RootCauseID in zip(df_event['Creation Time'], df_event["RottCauseID"]):
    
        #Get current event mapping error code and occuring requency in 30 minutes piece on current timesatmp
        error_slice = dataframe_strip(df_error,timestamp,timedelta(minutes=30))
        error_code = error_slice['ErrorCode'].value_counts().nlargest(1).keys().tolist()[0]
        in30 = error_slice['ErrorCode'].value_counts().nlargest(1).tolist()[0]
        
        #Get the "error_code" occuring requency in 20 minutes timeframe
        error_slice_in20 = dataframe_strip(df_error,timestamp,timedelta(minutes=20))
        in20 = len(error_slice_in20[error_slice_in20['ErrorCode']== error_code])
        
        #Get the "error_code" occuring requency in 10 minutes timeframe
        error_slice_in10 = dataframe_strip(df_error,timestamp,timedelta(minutes=10))
        in10 = len(error_slice_in10[error_slice_in10['ErrorCode']== error_code])
        
        #append to output dataframe df_occurs_when of current event mapping error code and its requency
        df_occurs_when = df_occurs_when.append({'~id': 'occurs_when_{}'.format(id),'~from' : error_code,'~to' : RootCauseID,'~label':'occurs_when' , 'in30:Integer' : in30,'in20:Integer' : in20, 'in10:Integer' : in10},
        ignore_index = True)
        id = id + 1
        
    return df_occurs_when

## Excute transfomation logic and output to target S3 folder 

In [9]:
#Excute the Mapping logic with df_error and df_event
output_df_occurs_when = mapping(df_error,df_event)
display(output_df_occurs_when)

,~id,~from,~to,~label,in30:Integer,in20:Integer,in10:Integer
0,occurs_when_1,30334,rc1,occurs_when,2,1,0
1,occurs_when_2,30376,rc2,occurs_when,3,3,1
2,occurs_when_3,30470,rc3,occurs_when,2,2,2
3,occurs_when_4,30495,rc4,occurs_when,2,1,0
4,occurs_when_5,30370,rc5,occurs_when,2,1,0
...,...,...,...,...,...,...,...
126,occurs_when_127,30268,rc127,occurs_when,3,3,2
127,occurs_when_128,30408,rc128,occurs_when,3,2,1
128,occurs_when_129,30409,rc129,occurs_when,2,1,0
129,occurs_when_130,30566,rc130,occurs_when,3,2,2


In [10]:
#Ouptput to Rawdata Folder in out S3 Bucket for nepotune to consume
output_df_occurs_when.to_csv(output_location,index=False)